In [1]:
import numpy as np
%matplotlib inline
from pylab import *

import scipy.integrate
import scipy




from collections import namedtuple

import numpy as np
%matplotlib inline
from pylab import *

import astropy
from astropy import units as ur
from astropy import constants as cc
import astropy.modeling.blackbody



## Infrastructure


Some approximations
$\Omega \approx \pi \sin^2 \theta$ and this is equivalent to $\Omega \approx \frac{\pi}{(2\cdot f/\#)^2}$

$G_{telescope} = \Omega_{telescope} \cdot A_{telescope} \cdot \epsilon_{telescope} $

In [2]:
def theta_to_omega(theta_rad):
    
    return np.pi * np.sin(theta_rad)**2 * ur.steradian

def fnum_to_omega(fnum):
    return np.pi /(2*fnum)**2 * ur.steradian

In [17]:
# Useful constants
nm = ur.nm
mm = ur.millimeter
µm = ur.micrometer
sr = ur.steradian
hc = cc.h * cc.c



# For the system dictionary
leff = "leff" # Effective wavelength
efficiency = "efficiency" # Total efficiency of spectrograph slit - to - detector
bandwidth = "bandwidth" # Slit width of spectrrograph in linear units
pixel_pitch = "pixel_pitch" # Distance of pixels in micron 
focal_ratio = "focal_ratio" # f/# of the spectrograph camera
agl = "agl" # Airplane altitude
ground_speed = "ground_speed" # ground speed
objective_ø = "objective_ø" # Diameter of objective
nsamp = "nsamp" # number of samples (GRDs) per resolution
rulingdensity = "rulingdensity" # Grating ruling density
lstart = "lstart" # Spectrum starting wavelength
lend = "lend" # Spectrum ending wavelength

In [4]:
Reflectivity_Canopy = 0.3 # This is the light reflected back up

Leff = 700 * nm
EPP = (hc/Leff).to(ur.erg) # Energy per photon

In [26]:
def print_system(system):
    """ pretty print the system """
    
    print()
    print("Aircraft")
    print("  Altitude: {0:5.0f}   Ground speed: {1:4.0f}".format(system[agl], system[ground_speed]))
    print("  *Exposure time: {0:+0.03f}".format(system_to_et(system)))
    
    print("Front end")
    print("  Objective diameter: {0:2.1f}".format(system[objective_ø]))
    print("  *GSD: {0:2.1f}   *GRD: {1:2.1f}".format(system_to_gsd(system), system_to_grd(system)))
    
    print("Spectrograph")
    print("  Effective wavelength %8s, thpt @ leff %5s" % (system[leff], system[efficiency]))
    print("  Bandwidth %8s" % (system[bandwidth]))
    print("  Focal ratio %3.1f" % system[focal_ratio])

    print("Detector")
    print("  Pixel pitch %8s   # samples %4.1f" % (system[pixel_pitch], system[nsamp]))
    print("  Spectrum is %4i pixels long in spectral direction (Prime has 1200 pix)" % (system_to_spectral_length(system)))
    
    print()
    print("* Derived")
    print("   GRASP: {0:4.1f}".format(system_to_grasp(system)))
    print("")
    

In [27]:
def system_to_grasp(system, loud=False):
    
    Omega = fnum_to_omega(system[focal_ratio])
    grasp = (system[pixel_pitch]**2 * Omega).to(µm**2 * sr) * system[efficiency]
    
    if loud: print("System grasp is %s" % grasp)
        
    return grasp

def system_to_fl(system):
    """ Focal length of system """
    
    return system[objective_ø] * system[focal_ratio]

def system_to_gsd(system):
    """ Ground sampling distance of system
    GSD is the pixel-to-pixel distance in meter
    nb GSD is not the same as ground resolution which will be several GSDs"""
    
    fl = system_to_fl(system)
    theta = system[pixel_pitch]/fl
    return (system[agl] * theta).to(ur.meter)

def system_to_grd(system):
    """ Ground resolution distance """
    
    return system_to_gsd(system)*system[nsamp]

def system_to_et(system):
    """ Time it takes airplane to move one slit distance
    (exposure time)"""
    
    dist = system_to_grd(system)
    speed = system[ground_speed]
    
    return (dist/speed).to(ur.second)

def system_to_spectral_length(system):
    ls, le = system[lstart], system[lend]
    
    dl = le-ls
    nr = dl/system[bandwidth]
    np = nr * system[nsamp]
    
    return np.cgs
    

def system_to_beam_size(system):
    
    def dtheta(system):
        """ 1.4 is from sin(45 degree) x 2"""
        density = system[rulingdensity]
        m=1 ; pitch = 1/density
        
        return system[bandwidth] /(1.4 * pitch) 
    
    dtheta = dtheta(system)
    
    focal_length = (system[pixel_pitch] * system[nsamp]) / dtheta
    
    return (focal_length/system[focal_ratio]).to(ur.mm)


Basic idea is that earth emits light at $L_\lambda$ that has units of $Power/Bandwidth/Etendue$.

Based on spectrograph parameters I compute $Etenude$ (or grasp, because efficiency is factored in). 

I can convert the Power in energy units to a count rate of photon by knowing the energy per photon (aka $epp$).


In [51]:


system ={leff: 720*nm, efficiency: 0.5, bandwidth: 0.18*nm, pixel_pitch: 11 * µm,
        focal_ratio: 2, agl: 1000*ur.meter, ground_speed: 60*ur.meter/ur.second,
        objective_ø: 5/3.*ur.mm, nsamp: 3.6, rulingdensity: 1800/ur.mm,
        lstart: 740*nm, lend: 780*nm}


print_system(system)
grasp = system_to_grasp(system)
f_lambda = Reflectivity_Canopy * 1300*ur.Watt/ur.meter**2/ur.micron/(2*np.pi*ur.sr) # http://www.powerfromthesun.net/Book/chapter02/chapter02.html

photon_per_second = (f_lambda / EPP * system[bandwidth] * grasp).cgs * system[nsamp]

print()
SW = (system_to_gsd(system) * 500).to(ur.meter)
print("Swath width at 500 pix %s" % (SW))
print("Ground resolution distance {0:3.2f} and is traversed in {1:1.4f} or {2:3.1f}".format(system_to_grd(system),
                                                                system_to_et(system),
                                                                     1/system_to_et(system)))

print()
NP = photon_per_second * system_to_et(system)
print("Number photons received per et %4.0f" % (NP))
print("Corresponding SNR pixel limit %4.0f:1 and slit limit %4.0f:1" % (np.sqrt(NP), 
                                                                    np.sqrt(NP)*np.sqrt(system[nsamp])))

print()
print("Beam Size")
print("  {0:3.0f}".format(system_to_beam_size(system)))


Aircraft
  Altitude:  1000 m   Ground speed:   60 m / s
  *Exposure time: +0.198 s
Front end
  Objective diameter: 1.7 mm
  *GSD: 3.3 m   *GRD: 11.9 m
Spectrograph
  Effective wavelength 720.0 nm, thpt @ leff   0.5
  Bandwidth  0.18 nm
  Focal ratio 2.0
Detector
  Pixel pitch  11.0 um   # samples  3.6
  Spectrum is  800 pixels long in spectral direction (Prime has 1200 pix)

* Derived
   GRASP: 11.9 sr um2


Swath width at 500 pix 1650.0000000000002 m
Ground resolution distance 11.88 m and is traversed in 0.1980 s or 5.1 1 / s

Number photons received per et 333374
Corresponding SNR pixel limit  577:1 and slit limit 1096:1

Beam Size
   86 mm


# SEE NOTEBOOK Page 3 Aug 2 2018

In [53]:


system ={leff: 720*nm, efficiency: 0.5, bandwidth: 0.18*nm, pixel_pitch: 11 * µm,
        focal_ratio: 2, agl: 1000*ur.meter, ground_speed: 60*ur.meter/ur.second,
        objective_ø: 5/3.*ur.mm, nsamp: 3.6, rulingdensity: 1800/ur.mm,
        lstart: 740*nm, lend: 780*nm}


print_system(system)
grasp = system_to_grasp(system)
f_lambda = Reflectivity_Canopy * 1300*ur.Watt/ur.meter**2/ur.micron/(2*np.pi*ur.sr) # http://www.powerfromthesun.net/Book/chapter02/chapter02.html

photon_per_second = (f_lambda / EPP * system[bandwidth] * grasp).cgs * system[nsamp]

print()
SW = (system_to_gsd(system) * 500).to(ur.meter)
print("Swath width at 500 pix %s" % (SW))
print("Ground resolution distance {0:3.2f} and is traversed in {1:1.4f} or {2:3.1f}".format(system_to_grd(system),
                                                                system_to_et(system),
                                                                     1/system_to_et(system)))

print()
NP = photon_per_second * system_to_et(system)
print("Number photons received per et %4.0f" % (NP))
print("Corresponding SNR pixel limit %4.0f:1 and slit limit %4.0f:1" % (np.sqrt(NP), 
                                                                    np.sqrt(NP)*np.sqrt(system[nsamp])))

print()
print("Beam Size")
print("  {0:3.0f}".format(system_to_beam_size(system)))


Aircraft
  Altitude:  1000 m   Ground speed:   60 m / s
  *Exposure time: +0.198 s
Front end
  Objective diameter: 1.7 mm
  *GSD: 3.3 m   *GRD: 11.9 m
Spectrograph
  Effective wavelength 720.0 nm, thpt @ leff   0.5
  Bandwidth  0.18 nm
  Focal ratio 2.0
Detector
  Pixel pitch  11.0 um   # samples  3.6
  Spectrum is  800 pixels long in spectral direction (Prime has 1200 pix)

* Derived
   GRASP: 11.9 sr um2


Swath width at 500 pix 1650.0000000000002 m
Ground resolution distance 11.88 m and is traversed in 0.1980 s or 5.1 1 / s

Number photons received per et 333374
Corresponding SNR pixel limit  577:1 and slit limit 1096:1

Beam Size
   86 mm
